# Image Data Preparation for Machine Learning
- The script iterates through directories within the dataset_dir, each representing a different class of images.
- Each image is opened, converted to RGB, resized to 128x128 pixels, and normalized (pixel values scaled to the range [0, 1]).
- The processed images are stored in a list, and corresponding labels (class names) are also recorded.
- These lists are then converted to numpy arrays for easy manipulation and use in machine learning models.


In [2]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

# Directory where the dataset is stored
dataset_dir = 'PlantVillage'

# Define image size for resizing
img_size = (128, 128)

# Initialize lists to hold image data and labels
images = []
labels = []

# Loop through each directory (class)
for class_name in os.listdir(dataset_dir):
    class_dir = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_dir):
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            if os.path.isfile(img_path):  # Ensure the path is a file
                try:
                    img = Image.open(img_path).convert('RGB')  # Convert image to RGB
                    img = img.resize(img_size)  # Resize image
                    img_array = np.array(img) / 255.0  # Normalize pixel values
                    images.append(img_array)
                    labels.append(class_name)
                except Exception as e:
                    print(f"Error loading image {img_path}: {e}")

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

print(f"Total images: {len(images)}")
print(f"Total labels: {len(labels)}")


Total images: 20638
Total labels: 20638


# Label Encoding 
- This script uses LabelEncoder from scikit-learn to convert categorical labels into numerical values, facilitating their use in machine learning algorithms.
- It prints out the mapping between class names and their corresponding encoded numerical labels, providing a reference for interpreting the encoded labels.

In [3]:
from sklearn.preprocessing import LabelEncoder

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Print class names and their corresponding labels
for class_name, label in zip(label_encoder.classes_, range(len(label_encoder.classes_))):
    print(f"{class_name}: {label}")


Pepper__bell___Bacterial_spot: 0
Pepper__bell___healthy: 1
Potato___Early_blight: 2
Potato___Late_blight: 3
Potato___healthy: 4
Tomato_Bacterial_spot: 5
Tomato_Early_blight: 6
Tomato_Late_blight: 7
Tomato_Leaf_Mold: 8
Tomato_Septoria_leaf_spot: 9
Tomato_Spider_mites_Two_spotted_spider_mite: 10
Tomato__Target_Spot: 11
Tomato__Tomato_YellowLeaf__Curl_Virus: 12
Tomato__Tomato_mosaic_virus: 13
Tomato_healthy: 14


# Dataset Splitting 
- The dataset is split into a training set (70%) and a remaining set (30%), which will be used for validation and testing. This ensures a substantial portion of the data is used for training the model.
- The remaining set is further divided equally into validation and test sets (15% each). The validation set is used for model tuning, while the test set is reserved for final evaluation.

In [4]:
X_train, X_rem, y_train, y_rem = train_test_split(images, labels_encoded, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_rem, y_rem, test_size=0.5, random_state=42)
print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(X_test)}")

# Data Augmentation and Generator Creation
- An ImageDataGenerator is created for the training set to perform various augmentations like rotations, shifts, shears, zooms, and flips. This helps improve the model's robustness by artificially increasing the diversity of the training data.
- The train_generator applies the augmentations to the training data, while the val_datagen is used without augmentations for the validation data, ensuring that the validation set remains unchanged for accurate evaluation.

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator()  # No augmentation for validation data

# Create generators
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
val_generator = val_datagen.flow(X_val, y_val, batch_size=32)


# Convolutional Neural Network (CNN) Model Definition and Training
- A Sequential CNN model is defined with three convolutional layers, each followed by max pooling. The model also includes a dense layer with 256 units and a final dense layer for classification with a softmax activation.
- The model is compiled with the Adam optimizer and sparse categorical cross-entropy loss, targeting accuracy as the evaluation metric.

In [7]:
import tensorflow as tf

# Define the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator
)

# Save the model
model.save('leaf_classification_model.keras')


c:\Users\Alex Kumar\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     3,211,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │         3,855 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,244,015 (12.37 MB)

 Trainable params: 3,244,015 (12.37 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20


c:\Users\Alex Kumar\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


452/452 ━━━━━━━━━━━━━━━━━━━━ 35s 74ms/step - accuracy: 0.3311 - loss: 2.0624 - val_accuracy: 0.6996 - val_loss: 0.9092
Epoch 2/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 34s 75ms/step - accuracy: 0.6926 - loss: 0.9401 - val_accuracy: 0.7426 - val_loss: 0.7963
Epoch 3/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 34s 74ms/step - accuracy: 0.7728 - loss: 0.6792 - val_accuracy: 0.7901 - val_loss: 0.6325
Epoch 4/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 35s 77ms/step - accuracy: 0.8127 - loss: 0.5327 - val_accuracy: 0.8698 - val_loss: 0.3803
Epoch 5/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 35s 76ms/step - accuracy: 0.8438 - loss: 0.4530 - val_accuracy: 0.8773 - val_loss: 0.3629
Epoch 6/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 33s 73ms/step - accuracy: 0.8587 - loss: 0.4100 - val_accuracy: 0.8850 - val_loss: 0.3153
Epoch 7/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 33s 73ms/step - accuracy: 0.8766 - loss: 0.3537 - val_accuracy: 0.9054 - val_loss: 0.2815
Epoch 8/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 33s 73ms/step - accuracy: 0.8895 - loss: 0.3221 - val_accurac

# Model Evaluation and Metrics Calculation
- The trained CNN model is evaluated on the test set to determine its accuracy. This provides an initial assessment of the model's performance on unseen data.
- Additional performance metrics, including precision, recall, and F1-score, are calculated to provide a comprehensive evaluation of the model. These metrics are weighted averages, considering the imbalance in class distribution.

In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy}")

# Get predictions
y_pred_probs = model.predict(X_test)
y_pred = y_pred_probs.argmax(axis=1)

# Calculate additional metrics
test_precision = precision_score(y_test, y_pred, average='weighted')
test_recall = recall_score(y_test, y_pred, average='weighted')
test_f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Test F1-Score: {test_f1}")

97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9387 - loss: 0.1796
Test Accuracy: 0.9344315528869629
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
Test Precision: 0.937954769611002
Test Recall: 0.9344315245478036
Test F1-Score: 0.9336201395628354
